Anshoo Mehra

Credit: Udacity

# Transfer Learning Quick Guide

Transfer learning involves taking a pre-trained neural network and adapting the neural network to a new, different data set.

Depending on both:

- the size of the new data set, and
- the similarity of the new data set to the original data set

The approach for using transfer learning will be different. There are four main cases:

- new data set is small, new data is similar to original training data - **End Of Covnet**
- new data set is small, new data is different from original training data **Fine Tune**
- new data set is large, new data is similar to original training data **Fine Tune & Re-Train**
- new data set is large, new data is different from original training data **Start of Covnet**

**End Of Covnet**
If the new data set is small and similar to the original training data:

- slice off the end of the neural network
- add a new fully connected layer that matches the number of classes in the new data set
- randomize the weights of the new fully connected layer; freeze all the weights from the pre-trained network
- train the network to update the weights of the new fully connected layer

To avoid overfitting on the small data set, the weights of the original network will be held constant rather than re-training the weights.

Since the data sets are similar, images from each data set will have similar higher level features. Therefore most or all of the pre-trained neural network layers already contain relevant information about the new data set and should be kept.

**Start of Covnet**

If the new data set is small and different from the original training data:

- slice off most of the pre-trained layers near the beginning of the network
- add to the remaining pre-trained layers a new fully connected layer that matches the number of classes in the new data set
- randomize the weights of the new fully connected layer; freeze all the weights from the pre-trained network
- train the network to update the weights of the new fully connected layer

Because the data set is small, overfitting is still a concern. To combat overfitting, the weights of the original neural network will be held constant, like in the first case.

But the original training set and the new data set do not share higher level features. In this case, the new network will only use the layers containing lower level features.

**Fine Tune**

If the new data set is large and similar to the original training data:

- remove the last fully connected layer and replace with a layer matching the number of classes in the new data set
- randomly initialize the weights in the new fully connected layer
- initialize the rest of the weights using the pre-trained weights
- re-train the entire neural network

Overfitting is not as much of a concern when training on a large data set; therefore, you can re-train all of the weights.

Because the original training set and the new data set share higher level features, the entire neural network is used as well.




**Fine Tune & Re-Train**

If the new data set is large and different from the original training data:

- remove the last fully connected layer and replace with a layer matching the number of classes in the new data set
- retrain the network from scratch with randomly initialized weights
- alternatively, you could just use the same strategy as the "large and similar" data case

Even though the data set is different from the training data, initializing the weights from the pre-trained network might make training faster. So this case is exactly the same as the case with a large, similar data set.

If using the pre-trained network as a starting point does not produce a successful model, another option is to randomly initialize the convolutional neural network weights and train the network from scratch.



## Things to Study : LeNet, AlexNet

http://yann.lecun.com/exdb/lenet/ (1998)

https://en.wikipedia.org/wiki/AlexNet (2012)

How do we apply transfer learning? Two popular methods are feature extraction and finetuning.

**Feature extraction.** Take a pretrained neural network and replace the final (classification) layer with a new classification layer, or perhaps even a small feedforward network that ends with a new classification layer. During training the weights in all the pre-trained layers are frozen, so only the weights for the new layer(s) are trained. In other words, the gradient doesn't flow backwards past the first new layer.

**Finetuning.** This is similar to feature extraction except the pre-trained weights aren't frozen. The network is trained end-to-end.

### AlexNet is trained Imagenet Database so let's see example of how well it is Identifying Animal Classes (Poodle, Weasle) from Caffee Database

In [4]:
import time
import tensorflow as tf
import numpy as np
from scipy.misc import imread
from caffe_classes import class_names
from alexnet import AlexNet


# placeholders
x = tf.placeholder(tf.float32, (None, 227, 227, 3))

# By keeping `feature_extract` set to `False`
# we indicate to keep the 1000 class final layer
# originally used to train on ImageNet.
probs = AlexNet(x, feature_extract=False)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# Read Images
im1 = (imread("poodle.png")[:, :, :3]).astype(np.float32)
im1 = im1 - np.mean(im1)

im2 = (imread("weasel.png")[:, :, :3]).astype(np.float32)
im2 = im2 - np.mean(im2)

# Run Inference
t = time.time()
output = sess.run(probs, feed_dict={x: [im1, im2]})

# Print Output
for input_im_ind in range(output.shape[0]):
    inds = np.argsort(output)[input_im_ind, :]
    print("Image", input_im_ind)
    for i in range(5):
        print("%s: %.3f" % (class_names[inds[-1 - i]], output[input_im_ind, inds[-1 - i]]))
    print()

print("Time: %.3f seconds" % (time.time() - t))

Image 0
miniature poodle: 0.389
toy poodle: 0.223
Bedlington terrier: 0.173
standard poodle: 0.150
komondor: 0.026

Image 1
weasel: 0.331
polecat, fitch, foulmart, foumart, Mustela putorius: 0.280
black-footed ferret, ferret, Mustela nigripes: 0.210
mink: 0.081
Arctic fox, white fox, Alopex lagopus: 0.027

Time: 0.126 seconds


### Now let's see if we apply past learning of ImageNet database on Traffic Signs how well it performs .. 

In [6]:
"""
The traffic signs are 32x32 so you
have to resize them to be 227x227 before
passing them to AlexNet.
"""
import time
import tensorflow as tf
import numpy as np
from scipy.misc import imread
from caffe_classes import class_names
from alexnet import AlexNet


# placeholders
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
resized = tf.image.resize_images(x, (227, 227))

probs = AlexNet(resized)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

# Read Images
im1 = imread("construction.jpg").astype(np.float32)
im1 = im1 - np.mean(im1)

im2 = imread("stop.jpg").astype(np.float32)
im2 = im2 - np.mean(im2)

# Run Inference
t = time.time()
output = sess.run(probs, feed_dict={x: [im1, im2]})

# Print Output
for input_im_ind in range(output.shape[0]):
    inds = np.argsort(output)[input_im_ind, :]
    print("Image", input_im_ind)
    for i in range(5):
        print("%s: %.3f" % (class_names[inds[-1 - i]], output[input_im_ind, inds[-1 - i]]))
    print()

print("Time: %.3f seconds" % (time.time() - t))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Image 0
screen, CRT screen: 0.051
digital clock: 0.041
laptop, laptop computer: 0.030
balance beam, beam: 0.027
parallel bars, bars: 0.023

Image 1
digital watch: 0.395
digital clock: 0.275
bottlecap: 0.115
stopwatch, stop watch: 0.104
combination lock: 0.086

Time: 0.106 seconds


### Offcourse it is not peforming well, so we will apply feature extraction ..

**Feature Extraction**
The problem is that AlexNet was trained on the ImageNet database, which has 1000 classes of images. You can see the classes in the caffe_classes.py file. None of those classes involves traffic signs.

In order to successfully classify our traffic sign images, you need to remove the final, 1000-neuron classification layer and replace it with a new, 43-neuron classification layer.

This is called feature extraction, because you're basically extracting the image features inferred by the penultimate layer, and passing these features to a new classification layer.

Your output will probably not precisely match the sample output below, since the output will depend on the (probably random) initialization of weights in the network. That being said, the output classes you see should be present in signnames.csv.

In [7]:
import time
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.misc import imread
from alexnet import AlexNet

sign_names = pd.read_csv('signnames.csv')
nb_classes = 43

x = tf.placeholder(tf.float32, (None, 32, 32, 3))
resized = tf.image.resize_images(x, (227, 227))

# Returns the second final layer of the AlexNet model,
# this allows us to redo the last layer specifically for 
# traffic signs model.
fc7 = AlexNet(resized, feature_extract=True)
shape = (fc7.get_shape().as_list()[-1], nb_classes)
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.nn.xw_plus_b(fc7, fc8W, fc8b)
probs = tf.nn.softmax(logits)

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

# Read Images
im1 = imread("construction.jpg").astype(np.float32)
im1 = im1 - np.mean(im1)

im2 = imread("stop.jpg").astype(np.float32)
im2 = im2 - np.mean(im2)

# Run Inference
t = time.time()
output = sess.run(probs, feed_dict={x: [im1, im2]})

# Print Output
for input_im_ind in range(output.shape[0]):
    inds = np.argsort(output)[input_im_ind, :]
    print("Image", input_im_ind)
    for i in range(5):
        print("%s: %.3f" % (sign_names.ix[inds[-1 - i]][1], output[input_im_ind, inds[-1 - i]]))
    print()

print("Time: %.3f seconds" % (time.time() - t))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Image 0
Traffic signals: 0.064
Speed limit (60km/h): 0.051
Speed limit (120km/h): 0.043
Wild animals crossing: 0.040
Stop: 0.040

Image 1
Roundabout mandatory: 0.065
Children crossing: 0.059
Speed limit (60km/h): 0.057
Double curve: 0.043
Speed limit (120km/h): 0.042

Time: 0.125 seconds


/Users/anmehra/Desktop/Anconda_Py2.7/anaconda/envs/IntroToTensorFlow/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


First, I figure out the shape of the final fully connected layer, in my opinion this is the trickiest part. To do that I have to figure out the size of the output from fc7. Since it's a fully connected layer I know it's shape will be 2D so the second (or last) element of the list will be the size of the output. fc7.get_shape().as_list()[-1] does the trick. I then combine this with the number of classes for the Traffic Sign dataset to get the shape of the final fully connected layer, shape = (fc7.get_shape().as_list()[-1], nb_classes). The rest of the code is just the standard way to define a fully connected in TensorFlow. Finally, I calculate the probabilities via softmax, probs = tf.nn.softmax(logits).

**Training the Feature Extractor**
The feature extractor you just created works, in the sense that data will flow through the network and result in predictions.

But the predictions aren't accurate, because you haven't yet trained the new classification layer.

In order to do that, you'll need to read in the training dataset and train the network.

Training AlexNet (even just the final layer!) can take a little while, so if you don't have a GPU, running on a subset of the data is a good alternative. As a point of reference one epoch over the training set takes roughly 53-55 seconds with a GTX 970.

In [ ]:
import pickle
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from alexnet import AlexNet

nb_classes = 43
epochs = 10
batch_size = 128

with open('./train.p', 'rb') as f:
    data = pickle.load(f)

X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], test_size=0.33, random_state=0)

features = tf.placeholder(tf.float32, (None, 32, 32, 3))
labels = tf.placeholder(tf.int64, None)
resized = tf.image.resize_images(features, (227, 227))

# Returns the second final layer of the AlexNet model,
# this allows us to redo the last layer for the traffic signs
# model.
fc7 = AlexNet(resized, feature_extract=True)
fc7 = tf.stop_gradient(fc7)
shape = (fc7.get_shape().as_list()[-1], nb_classes)
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.nn.xw_plus_b(fc7, fc8W, fc8b)

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
loss_op = tf.reduce_mean(cross_entropy)
opt = tf.train.AdamOptimizer()
train_op = opt.minimize(loss_op, var_list=[fc8W, fc8b])
init_op = tf.initialize_all_variables()

preds = tf.arg_max(logits, 1)
accuracy_op = tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))

def eval_on_data(X, y, sess):
    total_acc = 0
    total_loss = 0
    for offset in range(0, X.shape[0], batch_size):
        end = offset + batch_size
        X_batch = X[offset:end]
        y_batch = y[offset:end]

        loss, acc = sess.run([loss_op, accuracy_op], feed_dict={features: X_batch, labels: y_batch})
        total_loss += (loss * X_batch.shape[0])
        total_acc += (acc * X_batch.shape[0])

    return total_loss/X.shape[0], total_acc/X.shape[0]

with tf.Session() as sess:
    sess.run(init_op)

    for i in range(epochs):
        # training
        X_train, y_train = shuffle(X_train, y_train)
        t0 = time.time()
        for offset in range(0, X_train.shape[0], batch_size):
            end = offset + batch_size
            sess.run(train_op, feed_dict={features: X_train[offset:end], labels: y_train[offset:end]})

        val_loss, val_acc = eval_on_data(X_val, y_val, sess)
        print("Epoch", i+1)
        print("Time: %.3f seconds" % (time.time() - t0))
        print("Validation Loss =", val_loss)
        print("Validation Accuracy =", val_acc)
        print("")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `argmax` instead
Epoch 1
Time: 1098.138 seconds
Validation Loss = 0.513324991621
Validation Accuracy = 0.867145838187

Epoch 2
Time: 1223.846 seconds
Validation Loss = 0.353884641375
Validation Accuracy = 0.909807558539

Epoch 3
Time: 1052.179 seconds
Validation Loss = 0.263128438049
Validation Accuracy = 0.937166705305

Epoch 4
Time: 1070.928 seconds
Validation Loss = 0.21622725981
Validation Accuracy = 0.945590849393



**Even today AlexNet is very popular for computer vision. Though not all features were necessary, and hence there are various implementaion of AlexNet available today ..**

## Let's look at other Networks which came after Alexnet 

## In 2014, VGGNet & GoogleLeNet almost tied with ImageNet Competition...

** VGG ** has only 7.3% error classification ..

** GoogLeNet ** has only 6.7% error classification .. It uses INCEPTION MODULE ( See NN Repo, as quick refresher it is combination module of various layers, Conv and Pooling etc .. So, it is combination module of Avg Pool >> 1x1 >> 1x1 >> 3x3 >> 1x1 >> 5x5 >> at top concatanate ouput, this helps keep total # of params small, and it runs as fast as AlexNet with high-accuracy, so it is great choice for real-time classification like self driving car .. 

## In 2015, ResNet by Microsoft won combination .. 

**ResNet** it has loss of only 3% which is better than human accuracy

Let's try the CIFAR10 and GERMAN TRAFFIC SIGN DATABASES, before we go ahead, we will test classifier we just build on CIFAR10 and record our run as baseline .. 

### VGG Example .. Bottleneck Features are pre-trained to optimally run on CPU..

In [3]:
import pickle
import tensorflow as tf
import numpy as np
from keras.layers import Input, Flatten, Dense
from keras.models import Model

flags = tf.app.flags
FLAGS = flags.FLAGS

# command line flags
# flags.DEFINE_string('training_file', '', "Bottleneck features training file (.p)")
# flags.DEFINE_string('validation_file', '', "Bottleneck features validation file (.p)")
# flags.DEFINE_integer('epochs', 50, "The number of epochs.")
# flags.DEFINE_integer('batch_size', 256, "The batch size.")

def load_bottleneck_data(training_file, validation_file):
    """
    Utility function to load bottleneck features.

    Arguments:
        training_file - String
        validation_file - String
    """
    print("Training file", training_file)
    print("Validation file", validation_file)

    with open(training_file, 'rb') as f:
        train_data = pickle.load(f)
    with open(validation_file, 'rb') as f:
        validation_data = pickle.load(f)

    X_train = train_data['features']
    y_train = train_data['labels']
    X_val = validation_data['features']
    y_val = validation_data['labels']

    return X_train, y_train, X_val, y_val


def main(_):
    # load bottleneck data
    training_file = 'vgg-100/vgg_cifar10_100_bottleneck_features_train.p'
    validation_file = 'vgg-100/vgg_cifar10_bottleneck_features_validation.p'
    
    #Command Line 
    #X_train, y_train, X_val, y_val = load_bottleneck_data(FLAGS.training_file, FLAGS.validation_file)
    X_train, y_train, X_val, y_val = load_bottleneck_data(training_file, validation_file)

    print(X_train.shape, y_train.shape)
    print(X_val.shape, y_val.shape)

    nb_classes = len(np.unique(y_train))

    # define model
    input_shape = X_train.shape[1:]
    inp = Input(shape=input_shape)
    x = Flatten()(inp)
    x = Dense(nb_classes, activation='softmax')(x)
    model = Model(inp, x)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # train model
    model.fit(X_train, y_train, epochs=FLAGS.epochs, batch_size=FLAGS.batch_size, validation_data=(X_val, y_val), shuffle=True)


# parses flags and calls the `main` function above
if __name__ == '__main__':
    tf.app.run()

Training file vgg-100/vgg_cifar10_100_bottleneck_features_train.p
Validation file vgg-100/vgg_cifar10_bottleneck_features_validation.p
(1000, 1, 1, 512) (1000, 1)
(10000, 1, 1, 512) (10000, 1)
Train on 1000 samples, validate on 10000 samples
Epoch 1/50
1000/1000 [==============================] - 0s - loss: 5.4451 - acc: 0.0960 - val_loss: 4.3878 - val_acc: 0.1151
Epoch 2/50
1000/1000 [==============================] - 0s - loss: 4.1390 - acc: 0.1380 - val_loss: 3.8769 - val_acc: 0.1373
Epoch 3/50
1000/1000 [==============================] - 0s - loss: 3.7073 - acc: 0.1670 - val_loss: 3.4681 - val_acc: 0.1736
Epoch 4/50
1000/1000 [==============================] - 0s - loss: 3.2241 - acc: 0.2000 - val_loss: 3.0008 - val_acc: 0.2240
Epoch 5/50
1000/1000 [==============================] - 0s - loss: 2.7447 - acc: 0.2540 - val_loss: 2.6268 - val_acc: 0.2700
Epoch 6/50
1000/1000 [==============================] - 0s - loss: 2.3743 - acc: 0.3140 - val_loss: 2.3581 - val_acc: 0.3147
Epoch 7/

SystemExit: 

/Users/anmehra/Desktop/Anconda_Py2.7/anaconda/envs/IntroToTensorFlow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### ResNet Example .. Bottleneck Features are pre-trained to optimally run on CPU..

In [4]:
import pickle
import tensorflow as tf
import numpy as np
from keras.layers import Input, Flatten, Dense
from keras.models import Model

flags = tf.app.flags
FLAGS = flags.FLAGS

# command line flags
# flags.DEFINE_string('training_file', '', "Bottleneck features training file (.p)")
# flags.DEFINE_string('validation_file', '', "Bottleneck features validation file (.p)")
# flags.DEFINE_integer('epochs', 50, "The number of epochs.")
# flags.DEFINE_integer('batch_size', 256, "The batch size.")

def load_bottleneck_data(training_file, validation_file):
    """
    Utility function to load bottleneck features.

    Arguments:
        training_file - String
        validation_file - String
    """
    print("Training file", training_file)
    print("Validation file", validation_file)

    with open(training_file, 'rb') as f:
        train_data = pickle.load(f)
    with open(validation_file, 'rb') as f:
        validation_data = pickle.load(f)

    X_train = train_data['features']
    y_train = train_data['labels']
    X_val = validation_data['features']
    y_val = validation_data['labels']

    return X_train, y_train, X_val, y_val


def main(_):
    # load bottleneck data
    training_file = 'resnet-100/resnet_cifar10_100_bottleneck_features_train.p'
    validation_file = 'resnet-100/resnet_cifar10_bottleneck_features_validation.p'
    
    #Command Line 
    #X_train, y_train, X_val, y_val = load_bottleneck_data(FLAGS.training_file, FLAGS.validation_file)
    X_train, y_train, X_val, y_val = load_bottleneck_data(training_file, validation_file)

    print(X_train.shape, y_train.shape)
    print(X_val.shape, y_val.shape)

    nb_classes = len(np.unique(y_train))

    # define model
    input_shape = X_train.shape[1:]
    inp = Input(shape=input_shape)
    x = Flatten()(inp)
    x = Dense(nb_classes, activation='softmax')(x)
    model = Model(inp, x)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # train model
    model.fit(X_train, y_train, epochs=FLAGS.epochs, batch_size=FLAGS.batch_size, validation_data=(X_val, y_val), shuffle=True)


# parses flags and calls the `main` function above
if __name__ == '__main__':
    tf.app.run()

Training file resnet-100/resnet_cifar10_100_bottleneck_features_train.p
Validation file resnet-100/resnet_cifar10_bottleneck_features_validation.p
(1000, 1, 1, 2048) (1000, 1)
(10000, 1, 1, 2048) (10000, 1)
Train on 1000 samples, validate on 10000 samples
Epoch 1/50
1000/1000 [==============================] - 0s - loss: 2.5425 - acc: 0.1290 - val_loss: 2.1992 - val_acc: 0.2250
Epoch 2/50
1000/1000 [==============================] - 0s - loss: 1.9470 - acc: 0.3290 - val_loss: 1.7524 - val_acc: 0.4010
Epoch 3/50
1000/1000 [==============================] - 0s - loss: 1.5653 - acc: 0.4690 - val_loss: 1.4960 - val_acc: 0.4936
Epoch 4/50
1000/1000 [==============================] - 0s - loss: 1.2624 - acc: 0.6090 - val_loss: 1.3029 - val_acc: 0.5661
Epoch 5/50
1000/1000 [==============================] - 0s - loss: 1.0714 - acc: 0.6840 - val_loss: 1.1779 - val_acc: 0.6097
Epoch 6/50
1000/1000 [==============================] - 0s - loss: 0.9209 - acc: 0.7220 - val_loss: 1.0873 - val_acc: 0

SystemExit: 

/Users/anmehra/Desktop/Anconda_Py2.7/anaconda/envs/IntroToTensorFlow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### GoogLeNet (Inception) Example .. Bottleneck Features are pre-trained to optimally run on CPU..

In [5]:
import pickle
import tensorflow as tf
import numpy as np
from keras.layers import Input, Flatten, Dense
from keras.models import Model

flags = tf.app.flags
FLAGS = flags.FLAGS

# command line flags
# flags.DEFINE_string('training_file', '', "Bottleneck features training file (.p)")
# flags.DEFINE_string('validation_file', '', "Bottleneck features validation file (.p)")
# flags.DEFINE_integer('epochs', 50, "The number of epochs.")
# flags.DEFINE_integer('batch_size', 256, "The batch size.")

def load_bottleneck_data(training_file, validation_file):
    """
    Utility function to load bottleneck features.

    Arguments:
        training_file - String
        validation_file - String
    """
    print("Training file", training_file)
    print("Validation file", validation_file)

    with open(training_file, 'rb') as f:
        train_data = pickle.load(f)
    with open(validation_file, 'rb') as f:
        validation_data = pickle.load(f)

    X_train = train_data['features']
    y_train = train_data['labels']
    X_val = validation_data['features']
    y_val = validation_data['labels']

    return X_train, y_train, X_val, y_val


def main(_):
    # load bottleneck data
    training_file = 'inception-100/inception_cifar10_100_bottleneck_features_train.p'
    validation_file = 'inception-100/inception_cifar10_bottleneck_features_validation.p'
    
    #Command Line 
    #X_train, y_train, X_val, y_val = load_bottleneck_data(FLAGS.training_file, FLAGS.validation_file)
    X_train, y_train, X_val, y_val = load_bottleneck_data(training_file, validation_file)

    print(X_train.shape, y_train.shape)
    print(X_val.shape, y_val.shape)

    nb_classes = len(np.unique(y_train))

    # define model
    input_shape = X_train.shape[1:]
    inp = Input(shape=input_shape)
    x = Flatten()(inp)
    x = Dense(nb_classes, activation='softmax')(x)
    model = Model(inp, x)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # train model
    model.fit(X_train, y_train, epochs=FLAGS.epochs, batch_size=FLAGS.batch_size, validation_data=(X_val, y_val), shuffle=True)


# parses flags and calls the `main` function above
if __name__ == '__main__':
    tf.app.run()

Training file inception-100/inception_cifar10_100_bottleneck_features_train.p
Validation file inception-100/inception_cifar10_bottleneck_features_validation.p
(1000, 1, 1, 2048) (1000, 1)
(10000, 1, 1, 2048) (10000, 1)
Train on 1000 samples, validate on 10000 samples
Epoch 1/50
1000/1000 [==============================] - 0s - loss: 2.2689 - acc: 0.1740 - val_loss: 1.9632 - val_acc: 0.3038
Epoch 2/50
1000/1000 [==============================] - 0s - loss: 1.7616 - acc: 0.4060 - val_loss: 1.6507 - val_acc: 0.4399
Epoch 3/50
1000/1000 [==============================] - 0s - loss: 1.4270 - acc: 0.5700 - val_loss: 1.4468 - val_acc: 0.5214
Epoch 4/50
1000/1000 [==============================] - 0s - loss: 1.1967 - acc: 0.6620 - val_loss: 1.3126 - val_acc: 0.5647
Epoch 5/50
1000/1000 [==============================] - 0s - loss: 1.0349 - acc: 0.7070 - val_loss: 1.2248 - val_acc: 0.5916
Epoch 6/50
1000/1000 [==============================] - 0s - loss: 0.9155 - acc: 0.7400 - val_loss: 1.1646 

SystemExit: 

/Users/anmehra/Desktop/Anconda_Py2.7/anaconda/envs/IntroToTensorFlow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
